<a href="https://colab.research.google.com/github/mllopart/google-colab-notebooks/blob/main/phi_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Please run this notebook with a gpu (T4 at least)

In [1]:
!pip install sentencepiece transformers accelerate einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 3.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylin

In [2]:
import torch

In [3]:
torch.cuda.is_available()

True

In [4]:
from huggingface_hub import snapshot_download

In [5]:
repo_id = 'microsoft/Phi-3-mini-4k-instruct'

model_path = snapshot_download(repo_id=repo_id,repo_type="model", local_dir="./phi-3", local_dir_use_symlinks=False)

model_path = "./phi-2"

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/16.8k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

NOTICE.md:   0%|          | 0.00/1.77k [00:00<?, ?B/s]

LICENSE:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.85M [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.92k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

In [6]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

# Needs 12GB of vRAM to run in float32 (default)
# Run this line to load in float16. You need Gb of vRAM
torch.set_default_dtype(torch.float16)

model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto", trust_remote_code=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
model.device

device(type='cuda', index=0)

In [9]:
import time
def generate(prompt: str, generation_params: dict = {"max_length":200})-> str :
    s = time.time()

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, **generation_params)
    completion = tokenizer.batch_decode(outputs)[0]

    elapsed = time.time() - s

    num_input_tokens = inputs['input_ids'].shape[1]
    num_total_tokens = outputs.shape[1]
    num_output_tokens = float(num_total_tokens) - num_input_tokens
    speed = num_output_tokens / elapsed

    print(f"Took {round(elapsed,1)} seconds to generate {int(num_output_tokens)} new tokens at speed {round(speed, 1)} tokens/seconds")

    return completion

In [10]:
prompt = \
'''explain me the rerason of life'''

In [11]:
result = generate(prompt, generation_params={"max_length":200})
result

Took 9.2 seconds to generate 172 new tokens at speed 18.6 tokens/seconds


'<s> ```python\ndef print_prime(n):\n    """\n    print all primes between 1 and n\n    """\n    for i in range(2, n+1):\n        for j in range(2, i):\n            if i % j == 0:\n                break\n        else:\n            print(i)\n\nprint_prime(10)\n```\n\nThe above code works, but it\'s not very efficient. It has a time complexity of O(n^2). Can you optimize it to have a time complexity of O(n log log n)?\n\n\n### Response\n\nTo optimize the prime number printing function to have a time complexity of O(n log log n), we can use the Sieve of Eratosthenes algorithm. This algorithm efficiently finds all prime numbers up to a given limit, n, by iteratively marking the multiples of each prime number starting'

In [12]:
result = generate(prompt, generation_params={"max_length":200})
result

Took 7.5 seconds to generate 172 new tokens at speed 22.9 tokens/seconds


'<s> ```python\ndef print_prime(n):\n    """\n    print all primes between 1 and n\n    """\n    for i in range(2, n+1):\n        for j in range(2, i):\n            if i % j == 0:\n                break\n        else:\n            print(i)\n\nprint_prime(10)\n```\n\nThe above code works, but it\'s not very efficient. It has a time complexity of O(n^2). Can you optimize it to have a time complexity of O(n log log n)?\n\n\n### Response\n\nTo optimize the prime number printing function to have a time complexity of O(n log log n), we can use the Sieve of Eratosthenes algorithm. This algorithm efficiently finds all prime numbers up to a given limit, n, by iteratively marking the multiples of each prime number starting'

In [13]:
result = generate(prompt, generation_params={"max_length":2000})
result

Took 29.2 seconds to generate 556 new tokens at speed 19.1 tokens/seconds


'<s> ```python\ndef print_prime(n):\n    """\n    print all primes between 1 and n\n    """\n    for i in range(2, n+1):\n        for j in range(2, i):\n            if i % j == 0:\n                break\n        else:\n            print(i)\n\nprint_prime(10)\n```\n\nThe above code works, but it\'s not very efficient. It has a time complexity of O(n^2). Can you optimize it to have a time complexity of O(n log log n)?\n\n\n### Response\n\nTo optimize the prime number printing function to have a time complexity of O(n log log n), we can use the Sieve of Eratosthenes algorithm. This algorithm efficiently finds all prime numbers up to a given limit, n, by iteratively marking the multiples of each prime number starting from 2. The numbers which remain unmarked at the end are prime.\n\nHere\'s how you can implement it:\n\n```python\ndef print_primes(n):\n    """\n    Print all primes between 1 and n using the Sieve of Eratosthenes algorithm.\n    """\n    # Create a boolean array "prime[0..n]